In [7]:
import numpy as np
import os
from pathlib import Path
paths_list=[]
labels_list=[]
for i in range (1, 10):
    for j in range(0, 288):
        if os.path.isfile(f"Data sets 2a 0.5-2.5s 128Hz/data/A{str(i).rjust(2, '0')}TE{str(j).rjust(4, '0')}.npy"):
            paths_list.append(f"Data sets 2a 0.5-2.5s 128Hz/data/A{str(i).rjust(2, '0')}TE{str(j).rjust(4, '0')}.npy")
for i in range (1, 10):
    for j in range(0, 288):
        if os.path.isfile(f"Data sets 2a 0.5-2.5s 128Hz/data/A{str(i).rjust(2, '0')}EE{str(j).rjust(4, '0')}.npy"):
            paths_list.append(f"Data sets 2a 0.5-2.5s 128Hz/data/A{str(i).rjust(2, '0')}EE{str(j).rjust(4, '0')}.npy")
labels_list = np.load('Data sets 2a 0.5-2.5s 128Hz/labels_list.npy')
print(len(paths_list))
print(len(labels_list))
print(paths_list)
print(labels_list)

5184
5184
['Data sets 2a 0.5-2.5s 128Hz/data/A01TE0000.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0001.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0002.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0003.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0004.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0005.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0006.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0007.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0008.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0009.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0010.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0011.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0012.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0013.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0014.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0015.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0016.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0017.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE0018.npy', 'Data sets 2a 0.5-2.5s 128Hz/data/A01TE

In [22]:
from sklearn import model_selection
x_train, x_test_raw, y_train, y_test_raw = model_selection.train_test_split(
    paths_list, labels_list, test_size=0.2, random_state=42, shuffle=True
)
x_evaluate, x_test, y_evaluate, y_test = model_selection.train_test_split(
    x_test_raw, y_test_raw, test_size=0.5, random_state=42, shuffle=True
)

In [23]:
import tensorflow as tf
import math
class Datasets2aSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.x))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]

        return np.array([np.load(file_name) for file_name in batch_x]), np.array(batch_y)

In [24]:
from tensorflow.keras.layers import Conv2D,DepthwiseConv2D,AveragePooling2D,Activation,BatchNormalization,Dense,Dropout,Flatten,Input,SeparableConv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import optimizers
from keras.callbacks import ReduceLROnPlateau
num_classes = 4
def EEGNet():
    model = Sequential()
    clear_session()
    # Block 1
    model.add(Input((22,256,1)))
    model.add(Conv2D(8,(1, 64), strides=(1, 1), use_bias = False, padding='same'))
    model.add(BatchNormalization())
    model.add(DepthwiseConv2D((22, 1), use_bias = False, depth_multiplier = 2, depthwise_constraint = max_norm(1.)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(AveragePooling2D(1,4))
    model.add(Dropout(0.25))
    # Block 2
    model.add(SeparableConv2D(16,(1, 16), strides=(1, 1), use_bias = False, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(AveragePooling2D(1,8))
    model.add(Dropout(0.25))
    # Flatten & Dense
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax', kernel_constraint = max_norm(0.25)))
    model.compile('sgd', loss='categorical_crossentropy', metrics = ['categorical_accuracy'])
    return model
    
model = EEGNet()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 22, 256, 8)        512       
                                                                 
 batch_normalization (Batch  (None, 22, 256, 8)        32        
 Normalization)                                                  
                                                                 
 depthwise_conv2d (Depthwis  (None, 1, 256, 16)        352       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 1, 256, 16)        64        
 chNormalization)                                                
                                                                 
 activation (Activation)     (None, 1, 256, 16)        0         
                                                        

In [25]:
training_generator = Datasets2aSequence(x_train, y_train, 64)
validation_generator = Datasets2aSequence(x_test, y_test, 64)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.001)
model.fit(training_generator, epochs=500, validation_data=validation_generator, callbacks=[reduce_lr]) 

Epoch 1/500


2023-09-05 20:29:38.094438: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


65/65 [==============================] - 3s 30ms/step - loss: 1.3999 - categorical_accuracy: 0.2660 - val_loss: 1.3821 - val_categorical_accuracy: 0.2967 - lr: 0.0100
Epoch 2/500
65/65 [==============================] - 1s 20ms/step - loss: 1.3582 - categorical_accuracy: 0.3224 - val_loss: 1.3666 - val_categorical_accuracy: 0.3276 - lr: 0.0100
Epoch 3/500
65/65 [==============================] - 1s 21ms/step - loss: 1.3312 - categorical_accuracy: 0.3535 - val_loss: 1.3378 - val_categorical_accuracy: 0.3507 - lr: 0.0100
Epoch 4/500
65/65 [==============================] - 1s 22ms/step - loss: 1.3014 - categorical_accuracy: 0.3798 - val_loss: 1.3087 - val_categorical_accuracy: 0.3776 - lr: 0.0100
Epoch 5/500
65/65 [==============================] - 1s 22ms/step - loss: 1.2868 - categorical_accuracy: 0.3957 - val_loss: 1.2887 - val_categorical_accuracy: 0.3834 - lr: 0.0100
Epoch 6/500
65/65 [==============================] - 2s 24ms/step - loss: 1.2701 - categorical_accuracy: 0.4046 - val

In [27]:
model.evaluate(Datasets2aSequence(x_evaluate, y_evaluate, 64))

9/9 [==============================] - 1s 62ms/step - loss: 0.9105 - categorical_accuracy: 0.6390


[0.9104998111724854, 0.6389961242675781]

In [28]:
model.save('202309051955')

INFO:tensorflow:Assets written to: 202309051955/assets


INFO:tensorflow:Assets written to: 202309051955/assets
